In [280]:
# Re-import necessary libraries after execution state reset
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import networkx as nx
from colorama import Fore, Style, init
init(autoreset=True)

### 1. Degree Anonymization (Dynamic Programming & Greedy)

In [309]:
# Define the DegreeAnonymizer class
class DegreeAnonymizer:
    def __init__(self, degrees, k):
        self.degrees = sorted(degrees, reverse=True)
        self.n = len(degrees)
        self.k = k

    def dynamic_programming(self):
        n = self.n
        k = self.k
        dp = [float('inf')] * (n + 1)  # dp[i] = minimal cost for the first i nodes
        dp[0] = 0
        I = self._precompute_I()
        best_t = [-1] * (n + 1)  # Track the best split points for reconstruction

        for i in range(1, n + 1):
            if i < k:
                dp[i] = I[0][i-1]
                best_t[i] = 0  # Group [0, i-1]
            else:
                start = max(k, i - 2*k + 1)
                end = i - k + 1
                if start < end:
                    min_cost = float('inf')
                    best_split = start
                    for t in range(start, end):
                        current_cost = dp[t] + I[t][i-1]
                        if current_cost < min_cost:
                            min_cost = current_cost
                            best_split = t
                    dp[i] = min(dp[i], min_cost)
                    best_t[i] = best_split
                else:
                    dp[i] = I[0][i-1]
                    best_t[i] = 0  # Fallback to grouping all nodes [0, i-1]

        # Reconstruct the anonymized sequence using best_t
        anonymized = []
        current = n
        while current > 0:
            t = best_t[current]
            group_start = t
            group_end = current - 1  # Because current corresponds to i in dp[i]
            assigned_degree = max(self.degrees[group_start:group_end+1])  # Ensure maximum degree
            anonymized.extend([assigned_degree] * (group_end - group_start + 1))
            current = t
        anonymized.reverse()  # Reverse to match the original node order
        return anonymized

    def greedy(self):
        groups = []
        i = 0
        while i < self.n:
            current_degree = self.degrees[i]
            j = i

            # Expand the group until it has at least k elements
            while (j + 1 < self.n and (j - i + 1) < self.k):
                j += 1

            # Ensure we don't exceed the index limit
            j = min(j, self.n - 1)

            # Enforce 2k-1 group size restriction explicitly (optional, depending on your constraints)
            if (j - i + 1) > 2 * self.k - 1:
                j = i + 2 * self.k - 2

            # Set all degrees in this group to the maximum degree in the group
            max_deg = self.degrees[i]
            groups.append((i, j, max_deg))
            i = j + 1

        # Now construct the anonymized list clearly and safely
        anonymized = [0] * self.n  # Preallocate the anonymized list
        for start, end, deg in groups:
            for idx in range(start, end + 1):
                anonymized[idx] = deg

        return anonymized


    def _precompute_I(self):
        I = np.full((self.n, self.n), np.inf)
        for i in range(self.n):
            for j in range(i, self.n):
                group_size = j - i + 1
                if k <= group_size <= 2 * k - 1:  # Enforce group size restriction
                    max_degree = self.degrees[i]
                    cost = sum(max_degree - self.degrees[x] for x in range(i, j+1))
                    I[i][j] = cost
        return I

In [310]:
# Generate a random original graph (Barabasi-Albert model)
k = 3  # Ensure at least k-anonymity
num_nodes = 20
# edges_per_new_node = 5
# original_graph = nx.barabasi_albert_graph(num_nodes, edges_per_new_node)

p = 0.3  # Adjust probability to control density
original_graph = nx.erdos_renyi_graph(num_nodes, p)


# Extract the degree sequence of the original graph
original_degrees = sorted([d for _, d in original_graph.degree()], reverse=True)

# Initialize the DegreeAnonymizer
degree_anonymizer = DegreeAnonymizer(original_degrees, k)

# Perform degree anonymization using Dynamic Programming
anonymized_degrees_dp = degree_anonymizer.dynamic_programming()

# Perform degree anonymization using Greedy
anonymized_degrees_greedy = degree_anonymizer.greedy()

# Create DataFrames to display results
df_large_dp = pd.DataFrame({
    "Original Degrees": original_degrees,
    "Anonymized Degrees (DP)": anonymized_degrees_dp
})

df_large_greedy = pd.DataFrame({
    "Original Degrees": original_degrees,
    "Anonymized Degrees (Greedy)": anonymized_degrees_greedy
})

# Display results
print("Dynamic Programming Results:")
print(df_large_dp)

print("\nGreedy Results:")
print(df_large_greedy)

Dynamic Programming Results:
    Original Degrees  Anonymized Degrees (DP)
0                 11                       11
1                 11                       11
2                 10                       11
3                  9                        9
4                  9                        9
5                  8                        9
6                  8                        9
7                  7                        7
8                  7                        7
9                  7                        7
10                 6                        6
11                 6                        6
12                 6                        6
13                 6                        6
14                 6                        6
15                 5                        6
16                 4                        4
17                 4                        4
18                 4                        4
19                 4                        4

Gree

### 2. Graph Construction (Supergraph Algorithm)

In [311]:
class GraphConstructor:
    def __init__(self, G, anonymized_degrees):
        # Relabel nodes explicitly to integers (0...n-1)
        self.G = nx.convert_node_labels_to_integers(G, ordering='sorted')
        self.node_list = sorted(self.G.nodes())
        self.n = len(self.node_list)

        # Ensure anonymized_degrees match the node count
        if len(anonymized_degrees) != self.n:
            raise ValueError(f"Length mismatch: anonymized_degrees ({len(anonymized_degrees)}) and graph nodes ({self.n}) must match.")
        
        self.anonymized_degrees = anonymized_degrees

    def supergraph(self):
        # Explicit residual degree calculation
        residual = [
            self.anonymized_degrees[i] - self.G.degree(self.node_list[i])
            for i in range(self.n)
        ]

        # Check negative residuals
        if any(r < 0 for r in residual):
            print(Fore.RED + "[ERROR] Negative residual degrees detected.")
            return None

        edges = set(self.G.edges())

        while sum(residual) > 0:
            progress = False

            # Connect existing neighbors first
            for idx, v in enumerate(self.node_list):
                if residual[idx] <= 0:
                    continue

                candidates = [
                    u for u in self.G.neighbors(v)
                    if residual[self.node_list.index(u)] > 0
                    and (v, u) not in edges and (u, v) not in edges
                ]
                candidates.sort(key=lambda x: -residual[self.node_list.index(x)])

                for u in candidates:
                    u_idx = self.node_list.index(u)
                    if residual[idx] <= 0 or residual[u_idx] <= 0:
                        continue
                    edges.add((v, u))
                    residual[idx] -= 1
                    residual[u_idx] -= 1
                    progress = True

            # Then, connect non-neighbors if needed
            for idx, v in enumerate(self.node_list):
                while residual[idx] > 0:
                    candidates = [
                        u for u in self.node_list
                        if u != v and residual[self.node_list.index(u)] > 0
                        and not self.G.has_edge(v, u)
                        and (v, u) not in edges and (u, v) not in edges
                    ]
                    if not candidates:
                        print(Fore.RED + "[ERROR] No further edge additions possible.")
                        print(Fore.RED + f"[DEBUG] Residual degrees: {residual}")
                        return None
                    candidates.sort(key=lambda x: -residual[self.node_list.index(x)])
                    u = candidates[0]
                    u_idx = self.node_list.index(u)
                    edges.add((v, u))
                    residual[idx] -= 1
                    residual[u_idx] -= 1
                    progress = True

            if not progress:
                print(Fore.RED + "[ERROR] Supergraph construction stuck.")
                print(Fore.RED + f"[DEBUG] Residual degrees: {residual}")
                return None

        # Successfully construct the final supergraph
        G_super = nx.Graph()
        G_super.add_nodes_from(self.node_list)
        G_super.add_edges_from(edges)

        print(Fore.GREEN + "[INFO] Supergraph construction successful!")
        return G_super


### 3. Probing Scheme

In [ ]:
# def probing_scheme(G, k, anonymizer_func, max_iter=100):
#     # Step 0: Relabel nodes to integers 0, 1, ..., n-1 for consistency
#     G = nx.convert_node_labels_to_integers(G)
#     original_degrees = [d for _, d in G.degree()]
#     n = len(original_degrees)
#     print(Fore.CYAN + "\n--- Step 0: Initialization ---")
#     print(Fore.YELLOW + f"[DEBUG] Original graph has {n} nodes.")
#     print(Fore.YELLOW + f"[DEBUG] Original degrees (node: degree): { {i: original_degrees[i] for i in range(n)} }")

#     modified_nodes = set()

#     for iteration in range(max_iter):
#         print(Fore.CYAN + f"\n--- Iteration {iteration + 1}/{max_iter} ---")
#         print(Fore.YELLOW + f"[DEBUG] Current original degrees: {original_degrees}")

#         # Step 1: Anonymization
#         anonymizer = DegreeAnonymizer(original_degrees, k)
#         anonymized_degrees = anonymizer_func(anonymizer)
#         print(Fore.GREEN + "[DEBUG] Step 1: Anonymization Result:")
#         print(Fore.GREEN + f"[DEBUG] Anonymized degrees (node: degree): {anonymized_degrees}")

#         # Step 2: Graphicality check
#         is_graphical_flag = is_graphical(anonymized_degrees)
#         print(Fore.MAGENTA + f"[DEBUG] Step 2: Graphicality check result: {is_graphical_flag}")

#         if not is_graphical_flag:
#             print(Fore.RED + "[DEBUG] Step 3: Graphicality FAILED, adjusting degrees.")
#             sorted_nodes = sorted(range(n), key=lambda x: original_degrees[x])
#             print(Fore.RED + f"[DEBUG] Nodes sorted for adjustment: {sorted_nodes}")

#             node_to_adjust = None
#             for node in sorted_nodes:
#                 if node not in modified_nodes:
#                     node_to_adjust = node
#                     break

#             if node_to_adjust is None:
#                 print(Fore.RED + "[DEBUG] All nodes modified. Resetting modified_nodes set.")
#                 modified_nodes = set()
#                 node_to_adjust = sorted_nodes[0]

#             old_degree = original_degrees[node_to_adjust]
#             new_degree = min(old_degree + 1, n - 1)
#             original_degrees[node_to_adjust] = new_degree
#             modified_nodes.add(node_to_adjust)

#             print(Fore.RED + f"[DEBUG] Adjusted node {node_to_adjust}: degree {old_degree} → {new_degree}")
#             print(Fore.RED + f"[DEBUG] Modified nodes: {modified_nodes}")
#             continue

#         # Step 4: Graph construction
#         print(Fore.GREEN + "[DEBUG] Step 4: Graph construction attempt.")
#         constructor = GraphConstructor(G, anonymized_degrees)
#         G_anon = constructor.supergraph()

#         if G_anon is not None:
#             print(Fore.GREEN + "[DEBUG] Graph construction SUCCEEDED.")
#             print(Fore.GREEN + f"[DEBUG] Anonymized graph edges: {G_anon.edges()}")

#             # Step 5: Greedy_Swap optimization
#             print(Fore.BLUE + "[DEBUG] Step 5: Greedy_Swap optimization started.")
#             swapper = GreedySwap(G, G_anon)
#             G_anon_optimized = swapper.swap()
#             print(Fore.BLUE + "[DEBUG] Greedy_Swap optimization completed.")

#             # Edge overlap
#             original_edges = set(G.edges())
#             anonymized_edges = set(G_anon_optimized.edges())
#             edge_overlap = len(original_edges & anonymized_edges) / len(original_edges)
#             print(Fore.BLUE + f"[DEBUG] Edge overlap ratio: {edge_overlap:.2f}")

#             return G_anon_optimized

#         print(Fore.RED + "[DEBUG] Graph construction FAILED. Adjusting degrees again.")
        
#         sorted_nodes = sorted(range(n), key=lambda x: original_degrees[x])
#         print(Fore.RED + f"[DEBUG] Nodes sorted for secondary adjustment: {sorted_nodes}")

#         node_to_adjust = sorted_nodes[0]
#         old_degree = original_degrees[node_to_adjust]
#         new_degree = min(old_degree + 1, n - 1)
#         original_degrees[node_to_adjust] = new_degree
#         modified_nodes.add(node_to_adjust)

#         print(Fore.RED + f"[DEBUG] Adjusted node {node_to_adjust}: degree {old_degree} → {new_degree}")
#         print(Fore.RED + f"[DEBUG] Updated original degrees: {original_degrees}")

#     print(Fore.RED + "[DEBUG] --- Probing scheme FAILED after maximum iterations ---")
#     print(Fore.RED + "[DEBUG] Falling back to a complete graph with degrees n-1.")
#     return nx.complete_graph(n)

In [313]:
def probing_scheme(G, k, anonymizer_func, max_iter=100):
    # Step 0: Relabel nodes to integers 0, 1, ..., n-1 for consistency
    G = nx.convert_node_labels_to_integers(G)
    original_degrees = [d for _, d in G.degree()]
    n = len(original_degrees)
    print(Fore.CYAN + "\n--- Step 0: Initialization ---")
    print(Fore.YELLOW + f"[DEBUG] Original graph has {n} nodes.")
    print(Fore.YELLOW + f"[DEBUG] Original degrees (node: degree): { {i: original_degrees[i] for i in range(n)} }")

    modified_nodes = set()

    best_G_anon = None
    best_edge_overlap = 0

    for iteration in range(max_iter):
        print(Fore.CYAN + f"\n--- Iteration {iteration + 1}/{max_iter} ---")
        print(Fore.YELLOW + f"[DEBUG] Current original degrees: {original_degrees}")

        # Step 1: Anonymization
        anonymizer = DegreeAnonymizer(original_degrees, k)
        anonymized_degrees = anonymizer_func(anonymizer)
        print(Fore.GREEN + "[DEBUG] Step 1: Anonymization Result:")
        print(Fore.GREEN + f"[DEBUG] Anonymized degrees (node: degree): {anonymized_degrees}")

        # Step 2: Graphicality check
        is_graphical_flag = is_graphical(anonymized_degrees)
        print(Fore.MAGENTA + f"[DEBUG] Step 2: Graphicality check result: {is_graphical_flag}")

        if not is_graphical_flag:
            print(Fore.RED + "[DEBUG] Step 3: Graphicality FAILED, adjusting degrees.")
            sorted_nodes = sorted(range(n), key=lambda x: original_degrees[x])
            print(Fore.RED + f"[DEBUG] Nodes sorted for adjustment: {sorted_nodes}")

            node_to_adjust = None
            for node in sorted_nodes:
                if node not in modified_nodes:
                    node_to_adjust = node
                    break

            if node_to_adjust is None:
                print(Fore.RED + "[DEBUG] All nodes modified. Resetting modified_nodes set.")
                modified_nodes = set()
                node_to_adjust = sorted_nodes[0]

            old_degree = original_degrees[node_to_adjust]
            new_degree = min(old_degree + 1, n - 1)
            original_degrees[node_to_adjust] = new_degree
            modified_nodes.add(node_to_adjust)

            print(Fore.RED + f"[DEBUG] Adjusted node {node_to_adjust}: degree {old_degree} → {new_degree}")
            print(Fore.RED + f"[DEBUG] Modified nodes: {modified_nodes}")
            continue

        # Step 4: Graph construction
        print(Fore.GREEN + "[DEBUG] Step 4: Graph construction attempt.")
        constructor = GraphConstructor(G, anonymized_degrees)
        G_anon = constructor.supergraph()

        if G_anon is not None:
            print(Fore.GREEN + "[DEBUG] Graph construction SUCCEEDED.")
            print(Fore.GREEN + f"[DEBUG] Anonymized graph edges: {G_anon.edges()}")

            # Evaluate edge overlap
            edge_overlap = evaluate_edge_overlap(G, G_anon)
            print(Fore.BLUE + f"[DEBUG] Edge overlap ratio: {edge_overlap:.2f}")

            # Keep the graph with the highest edge overlap
            if edge_overlap > best_edge_overlap:
                best_G_anon = G_anon
                best_edge_overlap = edge_overlap

            # Stop if edge overlap is sufficiently high
            if edge_overlap >= 0.9:  # Example threshold
                print(Fore.GREEN + "[DEBUG] Edge overlap ratio is sufficiently high. Stopping early.")
                return G_anon

        print(Fore.RED + "[DEBUG] Graph construction FAILED. Adjusting degrees again.")

        sorted_nodes = sorted(range(n), key=lambda x: original_degrees[x])
        print(Fore.RED + f"[DEBUG] Nodes sorted for secondary adjustment: {sorted_nodes}")

        node_to_adjust = sorted_nodes[0]
        old_degree = original_degrees[node_to_adjust]
        new_degree = min(old_degree + 1, n - 1)
        original_degrees[node_to_adjust] = new_degree
        modified_nodes.add(node_to_adjust)

        print(Fore.RED + f"[DEBUG] Adjusted node {node_to_adjust}: degree {old_degree} → {new_degree}")
        print(Fore.RED + f"[DEBUG] Updated original degrees: {original_degrees}")

    print(Fore.RED + "[DEBUG] --- Probing scheme FAILED after maximum iterations ---")
    print(Fore.RED + "[DEBUG] Falling back to the best anonymized graph found.")
    return best_G_anon

In [314]:
def validate_k_anonymity(d_anon, k):
    groups = {}
    for degree in d_anon:
        groups[degree] = groups.get(degree, 0) + 1
    for count in groups.values():
        if count < k:
            return False
    return True

In [315]:
def evaluate_edge_overlap(G_original, G_anon):
    original_edges = set(G_original.edges())
    anonymized_edges = set(G_anon.edges())
    edge_intersect = len(original_edges & anonymized_edges) / len(original_edges)
    return edge_intersect

In [316]:
# Step 1: Define the anonymizer functions
def dp_anonymizer(anonymizer):
    return anonymizer.dynamic_programming()

def greedy_anonymizer(anonymizer):
    return anonymizer.greedy()

# Step 2: Call the Probing scheme with the desired anonymizer function
G_anon_dp = probing_scheme(original_graph, k, anonymizer_func=dp_anonymizer)  # Using Dynamic Programming
G_anon_greedy = probing_scheme(original_graph, k, anonymizer_func=greedy_anonymizer)  # Using Greedy

# Step 3: Display Results
if G_anon_dp is not None:
    print("Anonymization Successful with Dynamic Programming!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_dp.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_dp.edges())
else:
    print("Anonymization failed with Dynamic Programming after max iterations.")

if G_anon_greedy is not None:
    print("Anonymization Successful with Greedy!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_greedy.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_greedy.edges())
else:
    print("Anonymization failed with Greedy after max iterations.")


--- Step 0: Initialization ---
[DEBUG] Original graph has 20 nodes.
[DEBUG] Original degrees (node: degree): {0: 7, 1: 4, 2: 9, 3: 4, 4: 6, 5: 9, 6: 7, 7: 11, 8: 7, 9: 5, 10: 6, 11: 10, 12: 6, 13: 8, 14: 8, 15: 4, 16: 11, 17: 6, 18: 6, 19: 4}

--- Iteration 1/100 ---
[DEBUG] Current original degrees: [7, 4, 9, 4, 6, 9, 7, 11, 7, 5, 6, 10, 6, 8, 8, 4, 11, 6, 6, 4]
[DEBUG] Step 1: Anonymization Result:
[DEBUG] Anonymized degrees (node: degree): [11, 11, 11, 9, 9, 9, 9, 7, 7, 7, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4]
[DEBUG] Step 2: Graphicality check result: True
[DEBUG] Step 4: Graph construction attempt.
[ERROR] Negative residual degrees detected.
[DEBUG] Graph construction FAILED. Adjusting degrees again.
[DEBUG] Nodes sorted for secondary adjustment: [1, 3, 15, 19, 9, 4, 10, 12, 17, 18, 0, 6, 8, 13, 14, 2, 5, 11, 7, 16]
[DEBUG] Adjusted node 1: degree 4 → 5
[DEBUG] Updated original degrees: [7, 5, 9, 4, 6, 9, 7, 11, 7, 5, 6, 10, 6, 8, 8, 4, 11, 6, 6, 4]

--- Iteration 2/100 ---
[DEBUG] Curre

### 3. Greedy_Swap Algorithm

In [ ]:
class GreedySwap:
    def __init__(self, G_original, G_anon):
        self.G_original = G_original
        self.G_anon = G_anon
        self.common_edges = set(G_original.edges()) & set(G_anon.edges())

    def find_max_swap(self):
        max_gain = 0
        best_swap = None
        edges = list(self.G_anon.edges())
        sampled_edges = random.sample(edges, min(len(edges), int(len(edges) ** 0.5)))  # Sample O(√|Ê|) edges
        for i in range(len(sampled_edges)):
            for j in range(i + 1, len(sampled_edges)):
                e1, e2 = sampled_edges[i], sampled_edges[j]
                u1, v1 = e1
                u2, v2 = e2
                # Check swap validity
                if not self.G_anon.has_edge(u1, u2) and not self.G_anon.has_edge(v1, v2):
                    gain = 0
                    # Calculate gain in common edges
                    if (u1, u2) in self.G_original.edges() or (u2, u1) in self.G_original.edges():
                        gain += 2  # Higher weight for adding original edges
                    if (v1, v2) in self.G_original.edges() or (v2, v1) in self.G_original.edges():
                        gain += 2
                    # Subtract loss if original edges are removed
                    if e1 in self.common_edges or e2 in self.common_edges:
                        gain -= 1
                    if gain > max_gain:
                        max_gain = gain
                        best_swap = (e1, e2, (u1, u2), (v1, v2))
        return max_gain, best_swap

    def swap(self):
        while True:
            gain, swap = self.find_max_swap()
            if gain <= 0:
                break
            e1, e2, new_e1, new_e2 = swap
            # Perform swap
            self.G_anon.remove_edges_from([e1, e2])
            self.G_anon.add_edges_from([new_e1, new_e2])
            # Update common edges
            self.common_edges.discard(e1)
            self.common_edges.discard(e2)
            if self.G_original.has_edge(new_e1[0], new_e1[1]):
                self.common_edges.add(new_e1)
            if self.G_original.has_edge(new_e2[0], new_e2[1]):
                self.common_edges.add(new_e2)
        return self.G_anon

### 5. Evaluation Metrics

In [ ]:
def evaluate(G_original, G_anon):
    metrics = {}
    
    # L1 norm of degree differences
    l1 = sum(abs(G_original.degree(n) - G_anon.degree(n)) for n in G_original.nodes())
    metrics['L1'] = l1
    
    # Clustering Coefficient
    cc_orig = nx.average_clustering(G_original)
    cc_anon = nx.average_clustering(G_anon)
    metrics['CC_original'] = cc_orig
    metrics['CC_anon'] = cc_anon
    
    # Average Path Length (largest connected component)
    def largest_connected_component_apl(G):
        largest_cc = max(nx.connected_components(G), key=len)
        return nx.average_shortest_path_length(G.subgraph(largest_cc))
    
    apl_orig = largest_connected_component_apl(G_original)
    apl_anon = largest_connected_component_apl(G_anon)
    metrics['APL_original'] = apl_orig
    metrics['APL_anon'] = apl_anon
    
    # Edge Intersection
    if len(G_original.edges()) == 0:
        edge_intersect = 0
    else:
        edge_intersect = len(set(G_original.edges()) & set(G_anon.edges())) / len(G_original.edges())
    metrics['Edge_Intersection'] = edge_intersect
    
    return metrics

### 6. Usage

In [ ]:
# Generate a random original graph (Barabasi-Albert model)
k = 5  # Ensure at least k-anonymity
num_nodes = 20

# k = 3  # Ensure at least k-anonymity
# num_nodes = 200

p = 0.2  # Adjust probability to control density
original_graph = nx.erdos_renyi_graph(num_nodes, p)

# Step 1: Define the anonymizer functions
def dp_anonymizer(anonymizer):
    return anonymizer.dynamic_programming()

def greedy_anonymizer(anonymizer):
    return anonymizer.greedy()

# Step 2: Call the Probing scheme with the desired anonymizer function
G_anon_dp = probing_scheme(original_graph, k, anonymizer_func=dp_anonymizer)  # Using Dynamic Programming
G_anon_greedy = probing_scheme(original_graph, k, anonymizer_func=greedy_anonymizer)  # Using Greedy

# Step 3: Display Results
if G_anon_dp is not None:
    print("Anonymization Successful with Dynamic Programming!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_dp.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_dp.edges())
else:
    print("Anonymization failed with Dynamic Programming after max iterations.")

if G_anon_greedy is not None:
    print("Anonymization Successful with Greedy!")
    print("Original Degrees:", sorted([d for _, d in original_graph.degree()], reverse=True))
    print("Anonymized Degrees:", sorted([d for _, d in G_anon_greedy.degree()], reverse=True))
    print("Anonymized Graph Edges:", G_anon_greedy.edges())
else:
    print("Anonymization failed with Greedy after max iterations.")

# Evaluate results for Greedy
if G_anon_greedy is not None:
    metrics_greedy = evaluate(original_graph, G_anon_greedy)
    print("\nEvaluation Metrics (Greedy):", metrics_greedy)
else:
    print("\nGreedy Anonymization failed.")

# Evaluate results for Dynamic Programming
if G_anon_dp is not None:
    metrics_dp = evaluate(original_graph, G_anon_dp)
    print("\nEvaluation Metrics (Dynamic Programming):", metrics_dp)
else:
    print("\nDynamic Programming Anonymization failed.")


--- Step 0: Initialization ---
[DEBUG] Original graph has 200 nodes.
[DEBUG] Original degrees (node: degree): {0: 40, 1: 41, 2: 43, 3: 44, 4: 37, 5: 46, 6: 40, 7: 48, 8: 35, 9: 40, 10: 44, 11: 48, 12: 37, 13: 49, 14: 37, 15: 44, 16: 40, 17: 55, 18: 40, 19: 36, 20: 43, 21: 39, 22: 54, 23: 44, 24: 35, 25: 40, 26: 39, 27: 40, 28: 37, 29: 37, 30: 40, 31: 44, 32: 41, 33: 45, 34: 35, 35: 39, 36: 33, 37: 39, 38: 50, 39: 48, 40: 46, 41: 35, 42: 37, 43: 34, 44: 45, 45: 45, 46: 32, 47: 41, 48: 43, 49: 49, 50: 38, 51: 43, 52: 33, 53: 47, 54: 29, 55: 41, 56: 44, 57: 29, 58: 42, 59: 37, 60: 36, 61: 33, 62: 48, 63: 38, 64: 37, 65: 45, 66: 38, 67: 34, 68: 34, 69: 43, 70: 37, 71: 36, 72: 45, 73: 40, 74: 41, 75: 48, 76: 44, 77: 41, 78: 43, 79: 34, 80: 37, 81: 30, 82: 46, 83: 44, 84: 37, 85: 42, 86: 41, 87: 39, 88: 34, 89: 42, 90: 48, 91: 46, 92: 44, 93: 47, 94: 46, 95: 51, 96: 27, 97: 37, 98: 45, 99: 55, 100: 36, 101: 32, 102: 31, 103: 42, 104: 31, 105: 35, 106: 41, 107: 32, 108: 50, 109: 44, 110: 30,